amna rdifi

In [ ]:
cd /content/floraison

/content/floraison


In [ ]:
!pip install scrapy

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.4/44.4 kB 2.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 311.2/311.2 kB 17.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.2/3.2 MB 38.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 259.8/259.8 kB 14.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 104.9/104.9 kB 8.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.6/42.6 kB 2.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 74.6/74.6 kB 5.8 MB/s eta 0:00:00


In [ ]:
!scrapy

Scrapy 2.12.0 - no active project

Usage:
  scrapy <command> [options] [args]

Available commands:
  bench         Run quick benchmark test
  fetch         Fetch a URL using the Scrapy downloader
  genspider     Generate new spider using pre-defined templates
  runspider     Run a self-contained spider (without creating a project)
  settings      Get settings values
  shell         Interactive scraping console
  startproject  Create new project
  version       Print Scrapy version
  view          Open URL in browser, as seen by Scrapy

  [ more ]      More commands available when run from project directory

Use "scrapy <command> -h" to see more info about a command


In [ ]:
!scrapy startproject floraisonscraper

New Scrapy project 'floraisonscraper', using template directory '/usr/local/lib/python3.11/dist-packages/scrapy/templates/project', created in:
    /content/floraison/floraisonscraper

You can start your first spider with:
    cd floraisonscraper
    scrapy genspider example example.com


In [ ]:
cd /content/floraison/floraisonscraper/floraisonscraper/spiders

/content/floraison/floraisonscraper/floraisonscraper/spiders


In [ ]:
%ls

__init__.py


In [ ]:
!scrapy genspider floraison https://floraison.bio/en/

Created spider 'floraison' using template 'basic' in module:
  floraisonscraper.spiders.floraison


In [ ]:
!scrapy shell

2025-03-13 22:15:12 [scrapy.utils.log] INFO: Scrapy 2.12.0 started (bot: floraisonscraper)
2025-03-13 22:15:12 [scrapy.utils.log] INFO: Versions: lxml 5.3.1.0, libxml2 2.12.9, cssselect 1.3.0, parsel 1.10.0, w3lib 2.3.1, Twisted 24.11.0, Python 3.11.11 (main, Dec  4 2024, 08:55:07) [GCC 11.4.0], pyOpenSSL 24.2.1 (OpenSSL 3.3.2 3 Sep 2024), cryptography 43.0.3, Platform Linux-6.1.85+-x86_64-with-glibc2.35
2025-03-13 22:15:12 [scrapy.addons] INFO: Enabled addons:
[]
2025-03-13 22:15:12 [asyncio] DEBUG: Using selector: EpollSelector
2025-03-13 22:15:12 [scrapy.utils.log] DEBUG: Using reactor: twisted.internet.asyncioreactor.AsyncioSelectorReactor
2025-03-13 22:15:12 [scrapy.utils.log] DEBUG: Using asyncio event loop: asyncio.unix_events._UnixSelectorEventLoop
2025-03-13 22:15:12 [scrapy.utils.log] DEBUG: Using reactor: twisted.internet.asyncioreactor.AsyncioSelectorReactor
2025-03-13 22:15:12 [scrapy.utils.log] DEBUG: Using asyncio event loop: asyncio.unix_events._UnixSelectorEventLoop
20

fetch('https://floraison.bio/en')

response.css('div.center.collection__view-all a').attrib['href']

fetch('https://floraison.bio/en/collections/all-products')

response.css('h3.card__heading.h5 a').attrib['href']

fetch('https://floraison.bio/en/products/damascus-rose-hydrolate')

response.css('div.product__title h1::text').get()

response.css('span.price-item.price-item--regular::text').get()

response.css('div.product__description.rte.quick-add-hidden p').get()

response.css('div.product__description.rte.quick-add-hidden p[2]::text').get()
- response.xpath("//div[@class='product__description.rte.quick-add-hidden')]/p[2]/text()").get()
fetch('https://floraison.bio/en/collections/all-products')

produit=response.css('h3.card__heading.h5')

len(produit)

p=produit[1]

p.css('a').attrib['href']

fetch('')

In [ ]:
%%writefile /content/floraison/floraisonscraper/floraisonscraper/spiders/floraison.py
import scrapy


class FloraisonSpider(scrapy.Spider):
    name = "floraison"
    allowed_domains = ["floraison.bio"]
    start_urls = ["https://floraison.bio"]

    def parse(self, response):
      all_product=response.css('div.center.collection__view-all a::attr(href)').get()
      all_product_url='https://floraison.bio' + all_product
      yield response.follow(all_product_url,callback=self.parse_all_product)
    def parse_all_product(self,response):
      yield from self.parse_product(response)
      for i in range(2,13):
          next_page=response.url+'?page='+str(i)
          yield response.follow(next_page,callback=self.parse_product)
    def parse_product(self,response):
        products=response.css('h3.card__heading.h5')
        for p in products:
          url=p.css('a::attr(href)').get()
          yield response.follow(url,callback=self.parse_product_info)
    def parse_product_info(self,response):
        yield{
            'name':response.css('div.product__title h1::text').get(),
            'price':response.css('span.price-item.price-item--regular::text').get(),
            'description':response.css('div.product__description.rte.quick-add-hidden p::text').get(),
        }

Overwriting /content/floraison/floraisonscraper/floraisonscraper/spiders/floraison.py


In [ ]:
%cd /content/floraison/floraisonscraper


/content/floraison/floraisonscraper


In [ ]:
!scrapy crawl floraison -o data.csv

2025-03-13 22:21:57 [scrapy.utils.log] INFO: Scrapy 2.12.0 started (bot: floraisonscraper)
2025-03-13 22:21:57 [scrapy.utils.log] INFO: Versions: lxml 5.3.1.0, libxml2 2.12.9, cssselect 1.3.0, parsel 1.10.0, w3lib 2.3.1, Twisted 24.11.0, Python 3.11.11 (main, Dec  4 2024, 08:55:07) [GCC 11.4.0], pyOpenSSL 24.2.1 (OpenSSL 3.3.2 3 Sep 2024), cryptography 43.0.3, Platform Linux-6.1.85+-x86_64-with-glibc2.35
2025-03-13 22:21:57 [scrapy.addons] INFO: Enabled addons:
[]
2025-03-13 22:21:57 [asyncio] DEBUG: Using selector: EpollSelector
2025-03-13 22:21:57 [scrapy.utils.log] DEBUG: Using reactor: twisted.internet.asyncioreactor.AsyncioSelectorReactor
2025-03-13 22:21:57 [scrapy.utils.log] DEBUG: Using asyncio event loop: asyncio.unix_events._UnixSelectorEventLoop
2025-03-13 22:21:57 [scrapy.utils.log] DEBUG: Using reactor: twisted.internet.asyncioreactor.AsyncioSelectorReactor
2025-03-13 22:21:57 [scrapy.utils.log] DEBUG: Using asyncio event loop: asyncio.unix_events._UnixSelectorEventLoop
20